## Introduction
In this notebook, we will delve into the comprehensive exploration of the paper titled ["Anomaly Detection for Tabular Data with Internal Contrastive Learning."](https://openreview.net/forum?id=_hszZbt46bT) This paper introduces an innovative approach to anomaly detection by addressing the challenge of identifying out-of-class samples within tabular data, particularly when the data's structural characteristics are not well understood.

## Masking in ICL

The image below illustrates how ICL masks a feature vector for contrastive learning. The underlying learning problem. Given a **sample vector** $x_i$, they consider the **subvector** $a_i^3$ and its **complementary** $b_i^3$. The networks are trained to produce similar embeddings for this pair of vectors, while distancing the embedding of $a_i^{j^{\prime}}$ for $j^{\prime} \neq 3$ from that of $b_i^3$.

<center>
<img src="https://drive.google.com/uc?export=view&id=1WOfi9boxWG4ET3AKochRS8OaGtDjxmZe" width="500" aligh="center">
</center>


In [1]:
if "google.colab" in str(get_ipython()):  # type: ignore # noqa: F821
    %pip install copulas deepod pyod

In [2]:
import os
import warnings

import pandas as pd
from data_generator import DataGenerator
from myutils import Utils
from pyod_base import PYOD


warnings.filterwarnings(action="once")


## Dataset
In this part, we will work with the ["Outlier Detection DataSets (ODDS)"](https://odds.cs.stonybrook.edu/) dataset, a widely-used benchmark collection of datasets specifically designed for evaluating outlier detection algorithms. The ODDS dataset encompasses a diverse range of data types, structures, and characteristics, making it an ideal choice for assessing the effectiveness of anomaly detection methodologies. In the next cell we see all the datasets are reachable via ODDS.

In [3]:
datagenerator = DataGenerator()  # data generator
utils = Utils()  # utils function
os.listdir("datasets/Classical")


['18_Ionosphere.npz',
 '3_backdoor.npz',
 '42_WBC.npz',
 '35_SpamBase.npz',
 '47_yeast.npz',
 '26_optdigits.npz',
 '24_mnist.npz',
 '33_skin.npz',
 '16_http.npz',
 '43_WDBC.npz',
 '22_magic.gamma.npz',
 '28_pendigits.npz',
 '1_ALOI.npz',
 '5_campaign.npz',
 '32_shuttle.npz',
 '2_annthyroid.npz',
 '6_cardio.npz',
 '34_smtp.npz',
 '45_wine.npz',
 '12_fault.npz',
 '13_fraud.npz',
 '29_Pima.npz',
 '10_cover.npz',
 '9_census.npz',
 '36_speech.npz',
 '39_vertebral.npz',
 '15_Hepatitis.npz',
 '7_Cardiotocography.npz',
 '40_vowels.npz',
 '25_musk.npz',
 '41_Waveform.npz',
 '44_Wilt.npz',
 '46_WPBC.npz',
 '11_donors.npz',
 '21_Lymphography.npz',
 '17_InternetAds.npz',
 '31_satimage-2.npz',
 '8_celeba.npz',
 '27_PageBlocks.npz',
 '38_thyroid.npz',
 '30_satellite.npz',
 '4_breastw.npz',
 '37_Stamps.npz',
 '19_landsat.npz',
 '23_mammography.npz',
 '20_letter.npz',
 '14_glass.npz']

In [4]:
dataset_list = [
    "6_cardio",
    "13_fraud",
    "24_mnist",
    "42_WBC",
    "43_WDBC",
    "4_breastw",
    "25_musk",
    "40_vowels",
    "38_thyroid",
]  # choosing the datasets
model_dict = {"ICL": PYOD}  # choosing the model

# save the results
df_AUCROC = pd.DataFrame(data=None, index=dataset_list, columns=model_dict.keys())
df_AUCPR = pd.DataFrame(data=None, index=dataset_list, columns=model_dict.keys())


In [5]:
# seed for reproducible results
seed = 42

for dataset in dataset_list:
    datagenerator.dataset = dataset  # specify the dataset name
    data = datagenerator.generator(
        la=0.1, realistic_synthetic_mode=None, noise_type=None
    )  # only 10% labeled anomalies are available

    for name, model_cls in model_dict.items():
        # model initialization
        model = model_cls(seed=seed, model_name=name)

        # training, for unsupervised models the y label will be discarded
        model = model.fit(X_train=data["X_train"], y_train=data["y_train"])

        # output predicted anomaly score on testing set
        score = model.predict_score(data["X_test"])

        # evaluation
        result = utils.metric(y_true=data["y_test"], y_score=score)

        # save results
        df_AUCROC.loc[dataset, name] = result["aucroc"]


current noise type: None
{'Samples': 1831, 'Features': 21, 'Anomalies': 176, 'Anomalies Ratio(%)': 9.61}
best param: None
Start Training...
ensemble size: 4
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=19, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): M

testing: 100%|██████████| 9/9 [00:00<00:00, 292.19it/s]


subsampling for dataset 13_fraud...
current noise type: None
{'Samples': 10000, 'Features': 29, 'Anomalies': 16, 'Anomalies Ratio(%)': 0.16}
best param: None
Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=27, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(28, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True

testing: 100%|██████████| 47/47 [00:00<00:00, 337.64it/s]


current noise type: None
{'Samples': 7603, 'Features': 100, 'Anomalies': 700, 'Anomalies Ratio(%)': 9.21}
best param: None
Start Training...
ensemble size: 1
kernel size: 10
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=90, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(91, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(91, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(91, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net):

...

testing: 100%|██████████| 7/7 [00:00<00:00, 383.42it/s]


current noise type: None
{'Samples': 3772, 'Features': 6, 'Anomalies': 93, 'Anomalies Ratio(%)': 2.47}
best param: None
Start Training...
ensemble size: 8
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=4, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(

testing: 100%|██████████| 18/18 [00:00<00:00, 476.13it/s]


epoch  1, training loss: 21.067305, time: 0.3s
epoch 10, training loss: 0.737990, time: 0.3s
epoch 20, training loss: 0.385057, time: 0.2s
epoch 30, training loss: 0.329420, time: 0.3s
epoch 40, training loss: 0.295736, time: 0.2s
epoch 50, training loss: 0.328127, time: 0.3s
epoch 60, training loss: 0.231093, time: 0.2s
epoch 70, training loss: 0.239335, time: 0.2s
epoch 80, training loss: 0.238083, time: 0.2s
epoch 90, training loss: 0.222905, time: 0.3s
epoch100, training loss: 0.205139, time: 0.3s
Start Inference on the training data...


testing: 100%|██████████| 18/18 [00:00<00:00, 427.90it/s]


## Metrics and Results 
The Area Under the Receiver Operating Characteristic (AUROC) curve is a fundamental metric used to evaluate the performance of anomaly detection models. It provides insight into how well a model can distinguish between normal and anomalous instances across various threshold settings. AUC represents the degree or measure of separability calculated using ROC curves. It indicates how well the model can distinguish between classes. The higher the AUC, the better the model predicts 0 classes as 0 and 1 classes as 1. Below is a table that shows AUCROC values for different datasets.

In [6]:
df_AUCROC  # noqa: B018


,ICL
6_cardio,0.793364
13_fraud,0.867379
24_mnist,0.892911
42_WBC,0.952828
43_WDBC,0.974138
4_breastw,0.826051
25_musk,0.999109
40_vowels,0.850237
38_thyroid,0.864163
